In [1]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline
import math
from sklearn.metrics import mean_squared_error

import tensorflow as tf

In [2]:
os.getcwd()

'D:\\mywork\\DACON_CONTEST_2019\\SUHYUN'

In [3]:
os.chdir('D:\\mywork\\DACON_CONTEST_2019')

In [4]:
cluster = pd.read_csv('./SOOMIN/presentation/clusterIDs_k2.csv').T
cluster.columns = ['TEST_Cluster1', 'TEST_Cluster2', 'TRAIN_Cluster1', 'TRAIN_Cluster2']
cluster = cluster.iloc[1:]
cluster

,TEST_Cluster1,TEST_Cluster2,TRAIN_Cluster1,TRAIN_Cluster2
0,X10,X105,X11,X100
1,X102,X106,X113,X1000
2,X16,X107,X119,X1001
3,X18,X110,X12,X1002
4,X26,X111,X15,X1003
5,X30,X116,X2,X1004
6,X37,X120,X23,X1005
7,X41,X123,X27,X1006
8,X44,X124,X28,X1007
9,X5,X125,X40,X1008


In [5]:
np.random.seed(10)
test_c1 = np.random.choice(cluster['TEST_Cluster1'].dropna(), 5, replace=False)
test_c2 = np.random.choice(cluster['TEST_Cluster2'].dropna(), 5, replace=False)
train_c1 = np.random.choice(cluster['TRAIN_Cluster1'].dropna(), 10, replace=False)
train_c2 = np.random.choice(cluster['TRAIN_Cluster2'].dropna(), 10, replace=False)
print(test_c1, test_c2, train_c1, train_c2, sep='\n')

['X41' 'X50' 'X30' 'X37' 'X18']
['X231' 'X116' 'X149' 'X371' 'X218']
['X40' 'X84' 'X77' 'X93' 'X89' 'X15' 'X23' 'X60' 'X27' 'X6']
['X610' 'X1362' 'X476' 'X237' 'X1032' 'X1287' 'X428' 'X768' 'X989' 'X616']


In [6]:
raw_TRAIN = pd.read_csv('./SOOMIN/data/TRAIN_DHweightDaconArima_NAFilled.csv')
# raw_TRAIN = pd.read_csv('./DACON_DATASET/train.csv')
raw_TRAIN['Time'] = pd.to_datetime(raw_TRAIN['Time']) 
raw_TRAIN = raw_TRAIN.set_index('Time')
raw_TRAIN = raw_TRAIN.loc['2018-02-15':]

raw_TEST = pd.read_csv('./SOOMIN/data/TEST_DHweightDaconArima_NAFilled.csv')
# raw_TEST = pd.read_csv('./DACON_DATASET/test.csv')
raw_TEST['Time'] = pd.to_datetime(raw_TEST['Time']) 
raw_TEST = raw_TEST.set_index('Time')
raw_TEST = raw_TEST.loc['2018-02-15':]

raw_TEST

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
Time,,,,,,,,,,,,,,,,,,,,,
2018-02-15 00:00:00,1.088616,0.203000,0.462000,0.123000,0.153000,0.029000,0.194000,0.129000,0.008000,0.089000,...,0.466000,0.071000,0.378000,0.412000,0.638158,0.106000,0.104000,0.178,0.042,0.205000
2018-02-15 01:00:00,1.187378,0.563000,0.466000,0.308000,0.301000,0.155000,0.453000,0.358000,0.056000,0.130000,...,0.383000,0.057000,0.380000,0.406000,0.838203,0.191000,0.051000,0.182,0.043,0.218000
2018-02-15 02:00:00,1.260810,0.424000,0.428000,0.301000,0.213000,0.079000,0.388000,0.296000,0.093000,0.115000,...,0.360000,0.050000,0.402000,0.321000,0.959879,0.178000,0.134000,0.174,0.042,0.195000
2018-02-15 03:00:00,1.315409,0.501000,0.423000,0.326000,0.198000,0.079000,0.348000,0.280000,0.021000,0.219000,...,0.265000,0.058000,0.355000,0.314000,1.033986,0.168000,0.077000,0.175,0.043,0.202000
2018-02-15 04:00:00,1.356003,0.523000,0.562000,0.335000,0.170000,0.077000,0.426000,0.282000,0.027000,0.242000,...,0.246000,0.069000,0.289000,0.281000,1.079219,0.180000,0.090000,0.178,0.042,0.207000
2018-02-15 05:00:00,1.386187,0.527000,0.517000,0.476000,0.201000,0.073000,0.389000,0.305000,0.021000,0.206000,...,0.133000,0.048000,0.260000,0.340000,1.106927,0.169000,0.120000,0.176,0.041,0.212000
2018-02-15 06:00:00,1.408628,0.552000,0.372000,0.261000,0.178000,0.065000,0.365000,0.238000,0.022000,0.230000,...,0.130000,0.050000,0.425000,0.420000,1.123997,0.175000,0.091000,0.186,0.040,0.202000
2018-02-15 07:00:00,1.425314,0.523000,0.671000,0.363000,0.877000,0.066000,0.329000,0.261000,0.020000,0.449000,...,0.208000,0.141000,0.396000,0.346000,1.134610,0.376000,0.126000,0.116,0.041,0.209000
2018-02-15 08:00:00,1.437721,0.535000,0.383000,0.325000,0.593000,0.064000,0.620000,0.384000,0.021000,0.314000,...,0.186000,0.075000,0.385000,0.581000,1.141304,0.277000,0.097000,0.180,0.040,0.200000


In [7]:
train_group_set = raw_TRAIN.loc[:, train_c1.tolist()+train_c2.tolist()]
train_group_set 

,X40,X84,X77,X93,X89,X15,X23,X60,X27,X6,X610,X1362,X476,X237,X1032,X1287,X428,X768,X989,X616
Time,,,,,,,,,,,,,,,,,,,,
2018-02-15 00:00:00,10.592437,0.164,0.561,0.340,1.396,2.284564,2.691859,2.094527,4.322733,4.749095,0.066000,0.132000,0.091000,0.318000,0.277000,0.096000,0.020000,1.339000,0.113000,0.351000
2018-02-15 01:00:00,10.592437,0.161,0.486,0.344,1.394,2.284564,2.691859,2.094527,4.322733,4.749095,0.126000,0.278000,0.244000,0.202000,0.474000,0.338000,0.206000,0.599000,0.178000,1.495000
2018-02-15 02:00:00,10.592437,0.123,0.569,0.350,1.407,2.284564,2.691859,2.094527,4.322733,4.749095,0.227000,0.282000,0.252000,0.190000,0.302000,0.222000,0.134000,0.515000,0.223000,0.859000
2018-02-15 03:00:00,10.592437,0.115,0.590,0.333,1.396,2.284564,2.691859,2.094527,4.322733,4.749095,0.125000,0.286000,0.254000,0.163000,0.349000,0.282000,0.223000,0.458000,0.186000,0.492000
2018-02-15 04:00:00,10.592437,0.130,0.519,0.345,1.401,2.284564,2.691859,2.094527,4.322733,4.749095,0.135000,0.266000,0.251000,0.196000,0.305000,0.257000,0.266000,0.415000,0.212000,0.471000
2018-02-15 05:00:00,10.592437,0.113,0.517,0.347,1.396,2.284564,2.691859,2.094527,4.322733,4.749095,0.087000,0.292000,0.241000,0.198000,0.284000,0.258000,0.083000,0.475000,0.207000,0.374000
2018-02-15 06:00:00,10.592437,0.116,0.501,0.342,1.409,2.284564,2.691859,2.094527,4.322733,4.749095,0.104000,0.288000,0.423000,0.183000,0.313000,0.215000,0.089000,0.564000,0.290000,0.315000
2018-02-15 07:00:00,10.592437,0.114,0.556,0.337,1.395,2.284564,2.691859,2.094527,4.322733,4.749095,0.568000,0.615000,0.665000,0.150000,0.500000,0.218000,0.101000,0.479000,0.320000,0.355000
2018-02-15 08:00:00,10.592437,0.131,0.539,0.345,1.382,2.284564,2.691859,2.094527,4.322733,4.749095,0.677000,0.402000,0.916000,0.193000,0.352000,0.253000,0.335000,0.563000,0.828000,1.154000


In [8]:
test_group_set = raw_TEST.loc[:, test_c1.tolist()+test_c2.tolist()]
test_group_set

,X41,X50,X30,X37,X18,X231,X116,X149,X371,X218
Time,,,,,,,,,,
2018-02-15 00:00:00,2.371049,6.545218,4.803182,4.499756,2.495460,0.327000,0.199,0.261000,0.142000,0.361000
2018-02-15 01:00:00,2.334381,5.984280,4.529291,4.341709,2.500658,0.202000,0.168,0.391000,0.319000,0.241000
2018-02-15 02:00:00,2.302862,5.544196,4.305131,4.216720,2.504838,0.213000,0.123,0.248000,0.306000,0.284000
2018-02-15 03:00:00,2.275769,5.198929,4.121671,4.117873,2.508198,0.154000,0.113,0.215000,0.280000,0.295000
2018-02-15 04:00:00,2.252481,4.928049,3.971522,4.039701,2.510895,0.141000,0.167,0.211000,0.294000,0.267000
2018-02-15 05:00:00,2.232463,4.715530,3.848635,3.977879,2.513058,0.154000,0.182,0.219000,0.433000,0.347000
2018-02-15 06:00:00,2.215256,4.548798,3.748061,3.928988,2.514790,0.150000,0.226,0.214000,0.300000,0.706000
2018-02-15 07:00:00,2.200466,4.417989,3.665748,3.890323,2.516175,0.186000,0.115,0.266000,0.418000,0.514000
2018-02-15 08:00:00,2.187752,4.315363,3.598380,3.859746,2.517280,0.134000,0.115,0.404000,0.400000,0.675000


In [ ]:
train_all = pd.merge(train_group_set, test_group_set, left_index=True, right_index=True, how='outer')
train_all

,X40,X84,X77,X93,X89,X15,X23,X60,X27,X6,...,X41,X50,X30,X37,X18,X231,X116,X149,X371,X218
Time,,,,,,,,,,,,,,,,,,,,,
2018-02-15 00:00:00,10.592437,0.164,0.561,0.340,1.396,2.284564,2.691859,2.094527,4.322733,4.749095,...,2.371049,6.545218,4.803182,4.499756,2.495460,0.327000,0.199,0.261000,0.142000,0.361000
2018-02-15 01:00:00,10.592437,0.161,0.486,0.344,1.394,2.284564,2.691859,2.094527,4.322733,4.749095,...,2.334381,5.984280,4.529291,4.341709,2.500658,0.202000,0.168,0.391000,0.319000,0.241000
2018-02-15 02:00:00,10.592437,0.123,0.569,0.350,1.407,2.284564,2.691859,2.094527,4.322733,4.749095,...,2.302862,5.544196,4.305131,4.216720,2.504838,0.213000,0.123,0.248000,0.306000,0.284000
2018-02-15 03:00:00,10.592437,0.115,0.590,0.333,1.396,2.284564,2.691859,2.094527,4.322733,4.749095,...,2.275769,5.198929,4.121671,4.117873,2.508198,0.154000,0.113,0.215000,0.280000,0.295000
2018-02-15 04:00:00,10.592437,0.130,0.519,0.345,1.401,2.284564,2.691859,2.094527,4.322733,4.749095,...,2.252481,4.928049,3.971522,4.039701,2.510895,0.141000,0.167,0.211000,0.294000,0.267000
2018-02-15 05:00:00,10.592437,0.113,0.517,0.347,1.396,2.284564,2.691859,2.094527,4.322733,4.749095,...,2.232463,4.715530,3.848635,3.977879,2.513058,0.154000,0.182,0.219000,0.433000,0.347000
2018-02-15 06:00:00,10.592437,0.116,0.501,0.342,1.409,2.284564,2.691859,2.094527,4.322733,4.749095,...,2.215256,4.548798,3.748061,3.928988,2.514790,0.150000,0.226,0.214000,0.300000,0.706000
2018-02-15 07:00:00,10.592437,0.114,0.556,0.337,1.395,2.284564,2.691859,2.094527,4.322733,4.749095,...,2.200466,4.417989,3.665748,3.890323,2.516175,0.186000,0.115,0.266000,0.418000,0.514000
2018-02-15 08:00:00,10.592437,0.131,0.539,0.345,1.382,2.284564,2.691859,2.094527,4.322733,4.749095,...,2.187752,4.315363,3.598380,3.859746,2.517280,0.134000,0.115,0.404000,0.400000,0.675000


In [ ]:
train_all.to_csv('train_all.csv')

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
list_submission = []
look_back = 24
data_dim = 1
which_data_frame = train_all
#  앞의 숫자는 Unit 개수
# input_shape = (data_size, time_steps, features)  그러나 모델을 돌릴 땐, data_size는 따로 넘기지 않음.
# batch_input_shape = (data_size, time_steps, features) 주의점 data_size % batch_size == 0

# look_back = timesteip = seq_length
# data_dim ( 들어오는 데이터 열 개수)
# output_dim = 24 (hour: 나가게 하려는 값)

# convert nparray
for k in tqdm_notebook(range(len(which_data_frame.columns))):
    nparr = which_data_frame.iloc[:, k].to_frame()
    nparr.astype('float32')
    print(nparr)

    # normalization
    scaler = MinMaxScaler(feature_range=(0, 1))
    nptf = scaler.fit_transform(nparr)
    
    def create_dataset(dataset, look_back):
        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-1):
            a = dataset[i:(i + look_back)]
            dataX.append(a)
            dataY.append(dataset[i + look_back])
        return np.array(dataX), np.array(dataY)

    # create dataset for learning
    trainX, trainY = create_dataset(nptf, look_back)

    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (-1, look_back, data_dim))

    # simple lstm network learning
    model = Sequential()
    for i in range(2):
        model.add(LSTM(32, batch_input_shape=(1, look_back, data_dim), stateful=True, return_sequences=True))
        model.add(Dropout(0.5))
    model.add(LSTM(32, batch_input_shape=(1, look_back, data_dim), stateful=True))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    
    model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['accuracy'])
    MODEL_DIR = './model/'
    if not os.path.exists(MODEL_DIR):
        os.mkdir(MODEL_DIR)
    modelpath = "./model/[epoch:02d]-{val_loss:.4f}.hdf5"
    checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', vervose=1, save_best_only=True)
#     es = EarlyStopping(monitor = 'val_loss', patience = 15)
    
    history = model.fit(trainX, trainY, validation_split = 0.3, epochs=50, batch_size=1, verbose=2, callbacks = [checkpointer])
    
    y_vloss=history.history['val_loss']
    y_acc=history.history['acc']
    
    x_len = numpy.arange(len(y_acc))

    plt.plot(x_len, y_vloss, "o", c="red", markersize=3)
    plt.plot(x_len, y_acc, "o", c="blue", markersize=3)
    plt.show()

                           X40
Time                          
2018-02-15 00:00:00  10.592437
2018-02-15 01:00:00  10.592437
2018-02-15 02:00:00  10.592437
2018-02-15 03:00:00  10.592437
2018-02-15 04:00:00  10.592437
2018-02-15 05:00:00  10.592437
2018-02-15 06:00:00  10.592437
2018-02-15 07:00:00  10.592437
2018-02-15 08:00:00  10.592437
2018-02-15 09:00:00  10.592437
2018-02-15 10:00:00  10.592437
2018-02-15 11:00:00  10.592437
2018-02-15 12:00:00  10.592437
2018-02-15 13:00:00  10.592437
2018-02-15 14:00:00  10.592437
2018-02-15 15:00:00  10.592437
2018-02-15 16:00:00  10.592437
2018-02-15 17:00:00  10.592437
2018-02-15 18:00:00  10.592437
2018-02-15 19:00:00  10.592437
2018-02-15 20:00:00  10.592437
2018-02-15 21:00:00  10.592437
2018-02-15 22:00:00  10.592437
2018-02-15 23:00:00  10.592437
2018-02-16 00:00:00  10.592437
2018-02-16 01:00:00  10.592437
2018-02-16 02:00:00  10.592437
2018-02-16 03:00:00  10.592437
2018-02-16 04:00:00  10.592437
2018-02-16 05:00:00  10.592437
...     

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\callbacks.py:991: RuntimeWarning: invalid value encountered in less
  if self.monitor_op(current, self.best):


2267/2267 - 62s - loss: nan - acc: 4.4111e-04 - val_loss: nan - val_acc: 0.0000e+00
Epoch 2/50
2267/2267 - 73s - loss: nan - acc: 4.4111e-04 - val_loss: nan - val_acc: 0.0000e+00
Epoch 3/50
2267/2267 - 72s - loss: nan - acc: 4.4111e-04 - val_loss: nan - val_acc: 0.0000e+00
Epoch 4/50
2267/2267 - 75s - loss: nan - acc: 4.4111e-04 - val_loss: nan - val_acc: 0.0000e+00
Epoch 5/50
2267/2267 - 78s - loss: nan - acc: 4.4111e-04 - val_loss: nan - val_acc: 0.0000e+00
Epoch 6/50
2267/2267 - 85s - loss: nan - acc: 4.4111e-04 - val_loss: nan - val_acc: 0.0000e+00
Epoch 7/50


In [ ]:

    

    # make prediction
    testPredict = model.predict(testX)
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform(testY)
    testScore = math.sqrt(mean_squared_error(testY, testPredict))
    print('Train Score: %.2f RMSE' % testScore)

    # predict last value (or tomorrow?)
    lastX = nptf[-24:]
    lastX = np.reshape(lastX, (-1, look_back, data_dim))
    lastY_pred = model.predict(lastX)
    lastY = scaler.inverse_transform(lastY_pred)
    print('Predict the value of final day: %d' % lastY)  
    lastY_pred

    # predict last value (or tomorrow?)
    number = 1
    lastY_pred = 0
    for i in range(look_back):
        lastX = nptf[-look_back: ]
        lastX = np.reshape(lastX, (-1, look_back, data_dim))
        lastY_pred = model.predict(lastX)
        lastY = (scaler.inverse_transform(lastY_pred))/1000
        nptf = np.concatenate((nptf, lastY_pred))

        print('Predict value of column number {}: {}'.format(k, lastY[0][0]))  # 데이터 입력 마지막 다음날 종가 예측
        globals()['y_pred_list{}'.format(k)].append(lastY[0][0])
    list_submission.append(globals()['y_pred_list{}'.format(k)])
        